In [1]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))

In [ ]:
# Configura el entorno de trabajo
%reload_ext autoreload
%autoreload 2

In [ ]:
from data import load_raw_data_v2
import pandas as pd

# Download data from 2024 to the current year

from_year = 2024
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data_v2(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

rides.describe()


File 2024-01 was already in local storage
File 2024-02 was already in local storage
File 2024-03 was already in local storage
File 2024-04 was already in local storage
File 2024-05 was already in local storage
File 2024-06 was already in local storage
File 2024-07 was already in local storage
File 2024-08 was already in local storage
File 2024-09 was already in local storage
File 2024-10 was already in local storage
File 2024-11 was already in local storage
File 2024-12 was already in local storage
File 2025-01 was already in local storage
File 2025-02 was already in local storage
2025-03 file is not available
2025-04 file is not available
2025-05 file is not available
2025-06 file is not available
2025-07 file is not available
2025-08 file is not available
2025-09 file is not available
2025-10 file is not available
2025-11 file is not available
2025-12 file is not available


,pickup_datetime,pickup_location_id
count,48222016,4.822202e+07
mean,2024-08-06 01:50:30.125478,1.642402e+02
min,2024-01-01 00:00:00,1.000000e+00
25%,2024-04-21 13:46:25.750000,1.320000e+02
50%,2024-08-07 16:25:55,1.610000e+02
75%,2024-11-19 15:55:48.250000,2.330000e+02
max,2025-02-28 23:59:59,2.650000e+02
std,NaN,6.445180e+01


In [4]:
print(f'{len(rides)} rows downloaded')

48222016 rows downloaded


In [ ]:
# transform to time series
from data import transform_to_time_series

ts_data = transform_to_time_series(rides)

Rellenando huecos por localización: 100%|██████████| 263/263 [00:00<00:00, 417.28it/s]


In [22]:
import os
from dotenv import load_dotenv
from paths import PARENT_DIR
import hopsworks

# login into project hopsworks

load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_PROJECT_NAME =  'taxo_demand'
HOPSWORKS_API_KEY = "9JzCqysXLMaEbsBJ.fhJy3yLKT2ZRAwbh2A3r25EeytIL7HdU1l9U4QwvS01zX81I1uISJN9FgmlKZqvS"

project = hopsworks.login(
    api_key_value=HOPSWORKS_API_KEY,

)

2025-04-29 18:50:06,377 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-29 18:50:06,382 INFO: Initializing external client
2025-04-29 18:50:06,383 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-04-29 18:50:07,683 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224869


In [23]:
# get the feature store project
feature_store = project.get_feature_store()

In [24]:
# define name and version features groups
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [ ]:
# # if you want delete the feature group
# fg = feature_store.get_feature_group(
#             name=FEATURE_GROUP_NAME,
#             version=FEATURE_GROUP_VERSION
#         )
# fg.delete()
# print(f"Versión {FEATURE_GROUP_VERSION} borrada.")

In [ ]:
# create feature group without stream
feature_group = feature_store.create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Hourly time series data",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
    stream=False,    # desactiva completamente Kafka
)

In [ ]:
# save the feature group
feature_group.insert(
    ts_data,
    #storage="offline",
    write_options={"wait_for_job": True}
)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1224869/fs/1212495/fg/1440292






































%6|1745945443.755|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/1]: ssl://51.161.81.188:9093/1: Disconnected (after 49997ms in state UP, 1 identical error(s) suppressed)
%6|1745945443.863|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/2]: ssl://51.161.81.208:9093/2: Disconnected (after 50106ms in state UP, 1 identical error(s) suppressed)


































































































%6|1745945493.966|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.80.189:9093/0]: ssl://51.161.80.189:9093/0: Disconnected (after 100208ms in state UP, 1 identical error(s) suppressed)
































































































%6|1745945544.284|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/1]: ssl://51.161.81.188:9093/1: Disconnected (after 49996ms in state UP, 1 identical error(s) suppressed)



















































Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1224869/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions
2025-04-29 18:53:11,287 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-04-29 18:53:14,447 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


%6|1745945594.606|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/bootstrap: Disconnected (after 49999ms in state UP)
%6|1745945687.401|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/bootstrap: Disconnected (after 50002ms in state UP)
%6|1745945737.726|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/bootstrap: Disconnected (after 50001ms in state UP)
%6|1745945788.076|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/bootstrap: Disconnected (after 50023ms in state UP)


2025-04-29 18:56:40,243 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-04-29 18:56:40,396 INFO: Waiting for log aggregation to finish.
2025-04-29 18:56:52,280 INFO: Execution finished successfully.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

%6|1745945838.456|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/2]: ssl://51.161.81.208:9093/2: Disconnected (after 50043ms in state UP, 1 identical error(s) suppressed)
%6|1745945888.787|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/bootstrap: Disconnected (after 50000ms in state UP, 1 identical error(s) suppressed)
%6|1745945939.357|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/1]: ssl://51.161.81.188:9093/1: Disconnected (after 50016ms in state UP, 1 identical error(s) suppressed)
%6|1745945989.498|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/2]: ssl://51.161.81.208:9093/2: Disconnected (after 100377ms in state UP, 1 identical error(s) suppressed)
%6|1745946039.823|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.80.189:9093/0]: ssl://51.161.80.189:9093/0: Disconnected (after 50001ms in state UP, 1 identical error(s) suppressed)
%6|1745946090.145|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/bootstrap